In [68]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# 1st warm up : neural network
# https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#neural-networks

# https://pytorch.org/docs/stable/torch.html
# https://pytorch.org/docs/stable/autograd.html
# https://pytorch.org/docs/stable/nn.html

In [31]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,6,3)   # 对 1 sample * 6 channels * 3 height * 3 width 的输入应用二维卷积
        self.conv2 = nn.Conv2d(6,16,3)
        self.fc1 = nn.Linear(16*6*6,120) # 对传入数据应用线性变换：y=xA^T+b
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    
    def forward(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))    # 对激活后的函数做2d max池化
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = x.view(-1,self.num_flat_features(x))       # resize x 为一位数组 方便下一步线性变换
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self,x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

    
        

In [32]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [69]:
params = list(net.parameters())
print(params)
print(len(params))
print(params[0].size())

[Parameter containing:
tensor([[[[-0.2745, -0.1277, -0.2082],
          [ 0.0644,  0.2746, -0.2541],
          [ 0.0230, -0.2578, -0.0858]]],


        [[[-0.3323,  0.3309, -0.2447],
          [-0.0284,  0.3170,  0.1252],
          [ 0.1483, -0.0347, -0.2134]]],


        [[[ 0.3022,  0.1151,  0.2858],
          [ 0.3053, -0.0881, -0.1754],
          [ 0.0668,  0.2644,  0.1537]]],


        [[[-0.2849, -0.0462,  0.1934],
          [-0.1086, -0.2841,  0.2755],
          [ 0.2402,  0.2392,  0.1232]]],


        [[[ 0.2022,  0.3099,  0.2514],
          [-0.0503, -0.0478, -0.2278],
          [-0.1114,  0.0346, -0.3323]]],


        [[[-0.1701, -0.2107, -0.2377],
          [-0.0774, -0.2030, -0.0778],
          [-0.3244,  0.1019,  0.1465]]]], requires_grad=True), Parameter containing:
tensor([-0.1178, -0.2793, -0.2101,  0.0639,  0.0278, -0.2797],
       requires_grad=True), Parameter containing:
tensor([[[[ 0.1073, -0.0021, -0.0224],
          [-0.0808,  0.0549, -0.0980],
          [ 0.0091

        -0.0675,  0.1049], requires_grad=True)]
10
torch.Size([6, 1, 3, 3])


In [34]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[-0.0022, -0.0382, -0.0923, -0.0186,  0.0132,  0.0430, -0.0700, -0.1009,
         -0.0829,  0.1326]], grad_fn=<AddmmBackward>)


In [70]:
output = net(input)
target = torch.randn(10)
print(target)
target = target.view(1,-1)    # a.view(x,y) 将a转换为x行y列的tensor 参数不可为空 若为-1则该位置数据由其他位置推断
print(target)
criterion = nn.MSELoss()

loss = criterion(output,target)
print(loss)

tensor([-0.5250,  0.5543, -0.2118, -0.5746, -1.2649, -1.9040,  0.8082,  1.1861,
        -1.0621,  0.8983])
tensor([[-0.5250,  0.5543, -0.2118, -0.5746, -1.2649, -1.9040,  0.8082,  1.1861,
         -1.0621,  0.8983]])
tensor(0.7541, grad_fn=<MseLossBackward>)


In [38]:
# input -> conv2d 卷积函数 -> relu线性整流函数 激活函数 -> maxpool2d 池化层 -> conv2d 卷积函数 -> relu线性整流函数 激活函数 
#       -> maxpool2d 池化层  -> view resize -> linear 线性层 -> relu 激活函数 -> linear 线性层 -> relu 激活函数 -> linear 线性层
#       -> MSELoss 损失函数
#       ->loss 损失


In [39]:
print(loss.grad_fn)   # mseloss
print(loss.grad_fn.next_functions[0][0])   # linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) #relu

In [71]:
net.zero_grad()

print("conv1.bias.grad before backward")
print(net.conv1.bias.grad)

loss.backward()

print("conv1.bias.grad after backward")
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0098, -0.0080, -0.0280, -0.0296, -0.0088, -0.0096])


In [43]:
# weight = weight - learning_rate * gradient
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [67]:
import torch.optim as optim

# creat your optimizer
optimizer = optim.SGD(net.parameters(),lr=0.01)

# in your training loop
optimizer.zero_grad()   #zero the gradient buffers
output = net(input)
loss = criterion(output,target)
loss.backward()
optimizer.step()    # does the update 

print(loss)





tensor(0.6849, grad_fn=<MseLossBackward>)
